In [ ]:
# | default_exp templates

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

from fastcore.test import *
from jinja2 import UndefinedError

import jupyter_black
import nbdev.showdoc as showdoc

jupyter_black.load()

In [ ]:
# | export
from sal.core import Data
from typing import Any, Optional
from jinja2 import Environment, BaseLoader, Template, StrictUndefined
from pathlib import Path
from sal.frontmatter import FrontMatter

import abc

# Template relathed things

In [ ]:
# | exporti
# | hide
def _get_env() -> Environment:
    return Environment(loader=BaseLoader(), undefined=StrictUndefined)


# TODO start using a proper jinja template loader

    We need a rendering function capable of renderering a `template`, with `filters` and `context variables`...

In [ ]:
# | exporti
# | hide
def render_to_remove(
    template: str,  # template in string form
    filters: Optional[dict] = None,  # jinja filters
    **kwargs: Any,
) -> str:
    if not filters:
        filters = {}

    env = _get_env()
    env.filters.update(filters)

    jinja: Template = env.from_string(template)
    result: str = jinja.render(**kwargs)

    return result

In [ ]:
showdoc.show_doc(render_to_remove)

In [ ]:
# | hide
filters = {"upper": lambda n: n.upper()}
template = 'this is a template is this is my name "{{ name | upper }}" in upper case'
kwargs = dict(name="mauro")

result = render_to_remove(template, filters=filters, **kwargs)
expected = 'this is a template is this is my name "MAURO" in upper case'
test_eq(result, expected)

try:
    render_to_remove("{{ name }}")
except UndefinedError:
    pass

    Let's wrap up the rendering function from the `core` into an usable class

In [ ]:
# | exporti
class TemplateRenderer:
    def render(self, template: str | None = None, **kwargs: Any) -> str:
        if template is None:
            raise RuntimeError("Missing template")
        return render_to_remove(template, **kwargs)

## Template loading

We will need a way to get the templates

In [ ]:
# | exporti
class TemplateLoader(abc.ABC):
    def __init__(self):
        self.frontmatter_handler = FrontMatter()
    
    @abc.abstractmethod
    def get_template(self, name: str) -> str:
        """Separate method to allow an override to the template, before returning"""
        raise NotImplementedError


class MissingTemplate(Exception):
    def __init__(self, name: str):
        super().__init__(f"The template '{name}' is missing")
        self.name = name

In [ ]:
# | exporti
class InMemoryTemplateLoader(TemplateLoader):
    """
    Will keep a list of templates names + templates content
    """

    def __init__(
        self, *args: Any, templates: dict[str, str] | None = None, **kwargs: Any
    ) -> None:
        super().__init__(*args, **kwargs)
        self.templates: dict[str, str] = templates or {}

    def get_template_from_dict(self, name: str) -> str:
        if name in self.templates.keys():
            return self.templates[name]
        raise MissingTemplate(name)
        
    def _get_template(self, name: str, frontmatter: Optional[bool] = False) -> str:
        template = self.get_template_from_dict(name)  # type: ignore[safe-super]
        if not frontmatter:
            ret: str = self.frontmatter_handler.get_content(template)
            return ret
        ret2: str = self.frontmatter_handler.get_raw_frontmatter(template)
        return ret2

    def get_template(self, name: str) -> str:
        return self._get_template(name, frontmatter=False)

    def get_raw_frontmatter(self, name: str) -> str:
        return self._get_template(name, frontmatter=True)
    
    @classmethod
    def from_directory(cls, directory: str) -> "InMemoryTemplateLoader":
        path = Path(directory)
        glob = path.glob("*.jinja2")

        templates_raw = {}
        for p in glob:
            model_name = p.name.replace(".jinja2", "")
            with open(p, "r") as h:
                tpl = h.read()
            templates_raw[model_name] = tpl

        return cls(templates=templates_raw)    

## Tying rendering and loading together

And finally, put these 2 together to form a class to render a `Data` instance

In [ ]:
# | exporti
# TODO remove "any" typings


class Renderer:
    # if no template is passed in, we use the DEFAULT_TEMPLATE
    DEFAULT_TEMPLATE = "{% for child in children %}{{ child | render }}{% endfor %}"

    def __init__(
        self,
        *,
        renderer: TemplateRenderer,
        repository: TemplateLoader,
        filters: dict | None = None
    ):
        self.renderer = renderer
        self.repository = repository
        self.filters = filters or {}

    def render(self, data: Data, template: Optional[str] = None) -> str:
        if template is None:
            template = self.repository.get_template(data.name)

        ret = self.renderer.render(
            template=template,
            **data.attrs,
            filters={**self.filters, "render": self.render},
            node=data,
            children=data.children,
        )

        return ret

    def process(self, data: Data) -> str:
        return self.render(data)

    def get_template(self, *args, **kwargs) -> Any:
        return self.repository.get_template(*args, **kwargs)

    def get_metadata_for_template(self, path: str, data: Data) -> dict:
        template = self.repository.get_raw_frontmatter(path)  # type: ignore[call-arg]
        rendered = self.render(data, template)
        return self.repository.frontmatter_handler.parse(
            rendered
        )  # type: ignore[attr-defined]

The entry point for this class, after `__init__`, is the `process` method

---

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()